## Comparison of calculated vs. provided Angstrom coeffs (440-870) Aeronet V3

In [1]:
import pyaerocom as pya

VARS = ['ang4487aer', 'ang4487aer_calc', 'od550aer']
FILTER_NAME = 'WORLD-noMOUNTAINS'

2018-08-29 08:36:24,626:INFO:
Reading aliases ini file: /home/jonasg/repos/pyaerocom/pyaerocom/data/aliases.ini
/home/jonasg/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
2018-08-29 08:36:26,007:WARNING:
geopy library is not available. Aeolus data read not enabled


### Aeronet V3 Level 2

In [2]:
reader = pya.io.ReadUngridded()
reader.logger.setLevel('CRITICAL')
data = reader.read_dataset(pya.const.AERONET_SUN_V3L2_AOD_DAILY_NAME,
                           vars_to_retrieve=VARS)

2018-08-29 08:36:26,561:INFO:
Successfully loaded data for AeronetSunV3Lev2.daily from Cache


In [3]:
print(data)


Pyaerocom UngriddedData
-----------------------
Contains networks: ['AeronetSunV3Lev2.daily']
Contains variables: ['ang4487aer', 'ang4487aer_calc', 'od550aer']
Contains instruments: []
Total no. of stations: 1165


In [ ]:
regfilter = pya.Filter(FILTER_NAME)
data = regfilter(data)

2018-08-29 08:36:26,769:DEBUG:
OrderedDict([('station_name', 'AOE_Baotou'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 40.851699999999994), ('stat_lon', 109.62879999999998), ('stat_alt', 1314.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:36:26,772:DEBUG:
OrderedDict([('station_name', 'ARM_Nainital'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 29.358830000000005), ('stat_lon', 79.45826999999998), ('stat_alt', 1943.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:36:26,773:DEBUG:
OrderedDict([('station_name', 'ARM_WAIS'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', -79.470556), ('stat_lon', -112.083333), ('stat_alt', 1799.0), ('ts_t

2018-08-29 08:36:27,852:INFO:
adding chunk, new array size ((103787, 11))
2018-08-29 08:36:27,853:DEBUG:
OrderedDict([('station_name', 'BMKG_GAW_PALU'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', -1.6503), ('stat_lon', 120.1829), ('stat_alt', 1449.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:36:27,921:INFO:
adding chunk, new array size ((107526, 11))
2018-08-29 08:36:27,979:INFO:
adding chunk, new array size ((111265, 11))
2018-08-29 08:36:28,047:INFO:
adding chunk, new array size ((115004, 11))
2018-08-29 08:36:28,101:INFO:
adding chunk, new array size ((116004, 11))
2018-08-29 08:36:28,103:DEBUG:
OrderedDict([('station_name', 'BSRN_BAO_Boulder'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 40.045183), ('stat_lon', -105.00603300000002), ('stat_alt', 1604.0), ('ts_type', 'daily'), ('vari

2018-08-29 08:36:33,974:INFO:
adding chunk, new array size ((234702, 11))
2018-08-29 08:36:34,138:INFO:
adding chunk, new array size ((236695, 11))
2018-08-29 08:36:34,264:INFO:
adding chunk, new array size ((237695, 11))
2018-08-29 08:36:34,267:DEBUG:
OrderedDict([('station_name', 'Bose_Institute'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 27.031466999999996), ('stat_lon', 88.26140299999999), ('stat_alt', 2180.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:36:34,267:DEBUG:
OrderedDict([('station_name', 'Boulder'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 39.991465999999996), ('stat_lon', -105.26083199999998), ('stat_alt', 1659.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2

2018-08-29 08:36:44,429:INFO:
adding chunk, new array size ((368940, 11))
2018-08-29 08:36:44,929:INFO:
adding chunk, new array size ((369940, 11))
2018-08-29 08:36:45,172:INFO:
adding chunk, new array size ((370964, 11))
2018-08-29 08:36:45,554:INFO:
adding chunk, new array size ((371988, 11))
2018-08-29 08:36:45,777:INFO:
adding chunk, new array size ((373012, 11))
2018-08-29 08:36:45,779:DEBUG:
OrderedDict([('station_name', 'Calern_OCA'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 43.75220699999999), ('stat_lon', 6.923152999999999), ('stat_alt', 1270.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:36:45,981:INFO:
adding chunk, new array size ((374012, 11))
2018-08-29 08:36:46,279:INFO:
adding chunk, new array size ((375089, 11))
2018-08-29 08:36:46,923:INFO:
adding chunk, new array size ((376166, 11))
2018-08-29 08:36:47,29

2018-08-29 08:37:11,414:INFO:
adding chunk, new array size ((498031, 11))
2018-08-29 08:37:11,752:INFO:
adding chunk, new array size ((499031, 11))
2018-08-29 08:37:11,754:DEBUG:
OrderedDict([('station_name', 'DRAGON_Fort_Collins'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 40.592795), ('stat_lon', -105.14141), ('stat_alt', 1572.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:37:12,056:INFO:
adding chunk, new array size ((500031, 11))
2018-08-29 08:37:12,348:INFO:
adding chunk, new array size ((501031, 11))
2018-08-29 08:37:12,612:INFO:
adding chunk, new array size ((502031, 11))
2018-08-29 08:37:12,897:INFO:
adding chunk, new array size ((503031, 11))
2018-08-29 08:37:13,140:INFO:
adding chunk, new array size ((504031, 11))
2018-08-29 08:37:13,385:INFO:
adding chunk, new array size ((505031, 11))
2018-08-29 08:37:13,386:DEBU

...

2018-08-29 08:41:26,804:INFO:
adding chunk, new array size ((1164628, 11))
2018-08-29 08:41:27,684:INFO:
adding chunk, new array size ((1165628, 11))
2018-08-29 08:41:28,440:INFO:
adding chunk, new array size ((1167068, 11))
2018-08-29 08:41:29,324:INFO:
adding chunk, new array size ((1168508, 11))
2018-08-29 08:41:30,060:INFO:
adding chunk, new array size ((1169948, 11))
2018-08-29 08:41:30,668:INFO:
adding chunk, new array size ((1170948, 11))
2018-08-29 08:41:31,351:INFO:
adding chunk, new array size ((1171948, 11))
2018-08-29 08:41:32,167:INFO:
adding chunk, new array size ((1174000, 11))
2018-08-29 08:41:32,930:INFO:
adding chunk, new array size ((1176052, 11))
2018-08-29 08:41:33,679:INFO:
adding chunk, new array size ((1178104, 11))
2018-08-29 08:41:34,399:INFO:
adding chunk, new array size ((1179104, 11))
2018-08-29 08:41:35,451:INFO:
adding chunk, new array size ((1180104, 11))
2018-08-29 08:41:36,264:INFO:
adding chunk, new array size ((1181428, 11))
2018-08-29 08:41:36,988:I

2018-08-29 08:42:00,857:INFO:
adding chunk, new array size ((1258110, 11))
2018-08-29 08:42:01,560:INFO:
adding chunk, new array size ((1259451, 11))
2018-08-29 08:42:02,257:INFO:
adding chunk, new array size ((1260792, 11))
2018-08-29 08:42:02,894:INFO:
adding chunk, new array size ((1261792, 11))
2018-08-29 08:42:03,576:INFO:
adding chunk, new array size ((1263816, 11))
2018-08-29 08:42:04,134:INFO:
adding chunk, new array size ((1265840, 11))
2018-08-29 08:42:04,683:INFO:
adding chunk, new array size ((1267864, 11))
2018-08-29 08:42:04,921:INFO:
adding chunk, new array size ((1269531, 11))
2018-08-29 08:42:05,200:INFO:
adding chunk, new array size ((1271198, 11))
2018-08-29 08:42:05,848:INFO:
adding chunk, new array size ((1272865, 11))
2018-08-29 08:42:06,514:INFO:
adding chunk, new array size ((1273865, 11))
2018-08-29 08:42:07,165:INFO:
adding chunk, new array size ((1274865, 11))
2018-08-29 08:42:07,840:INFO:
adding chunk, new array size ((1275865, 11))
2018-08-29 08:42:07,841:D

2018-08-29 08:42:28,574:INFO:
adding chunk, new array size ((1306533, 11))
2018-08-29 08:42:29,466:INFO:
adding chunk, new array size ((1307533, 11))
2018-08-29 08:42:29,468:DEBUG:
OrderedDict([('station_name', 'NEON_Sterling'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 40.461903), ('stat_lon', -103.02929700000001), ('stat_alt', 1372.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:42:30,588:INFO:
adding chunk, new array size ((1308533, 11))
2018-08-29 08:42:31,591:INFO:
adding chunk, new array size ((1309533, 11))
2018-08-29 08:42:33,112:INFO:
adding chunk, new array size ((1310533, 11))
2018-08-29 08:42:34,092:INFO:
adding chunk, new array size ((1311533, 11))
2018-08-29 08:42:35,043:INFO:
adding chunk, new array size ((1312533, 11))
2018-08-29 08:42:36,482:INFO:
adding chunk, new array size ((1313533, 11))
2018-08-29 08:42:

2018-08-29 08:43:18,817:INFO:
adding chunk, new array size ((1389262, 11))
2018-08-29 08:43:19,856:INFO:
adding chunk, new array size ((1392286, 11))
2018-08-29 08:43:21,347:INFO:
adding chunk, new array size ((1395310, 11))
2018-08-29 08:43:22,327:INFO:
adding chunk, new array size ((1396310, 11))
2018-08-29 08:43:23,492:INFO:
adding chunk, new array size ((1397310, 11))
2018-08-29 08:43:24,922:INFO:
adding chunk, new array size ((1400423, 11))
2018-08-29 08:43:26,521:INFO:
adding chunk, new array size ((1403536, 11))
2018-08-29 08:43:28,240:INFO:
adding chunk, new array size ((1406649, 11))
2018-08-29 08:43:30,083:INFO:
adding chunk, new array size ((1407688, 11))
2018-08-29 08:43:31,017:INFO:
adding chunk, new array size ((1408727, 11))
2018-08-29 08:43:31,685:INFO:
adding chunk, new array size ((1409766, 11))
2018-08-29 08:43:32,717:INFO:
adding chunk, new array size ((1411539, 11))
2018-08-29 08:43:33,475:INFO:
adding chunk, new array size ((1413312, 11))
2018-08-29 08:43:34,125:I

2018-08-29 08:44:26,752:INFO:
adding chunk, new array size ((1476866, 11))
2018-08-29 08:44:28,238:INFO:
adding chunk, new array size ((1477866, 11))
2018-08-29 08:44:28,240:DEBUG:
OrderedDict([('station_name', 'Rexburg_Idaho'), ('PI', 'Brent_Holben'), ('dataset_name', 'AeronetSunV3Lev2.daily'), ('stat_lat', 43.82011), ('stat_lon', -111.78301000000002), ('stat_alt', 1501.0), ('ts_type', 'daily'), ('variables', ['ang4487aer', 'ang4487aer_calc', 'od550aer']), ('instrument_name', None)]) does not match filter and will be ignored
2018-08-29 08:44:29,590:INFO:
adding chunk, new array size ((1478866, 11))
2018-08-29 08:44:30,977:INFO:
adding chunk, new array size ((1482035, 11))
2018-08-29 08:44:32,331:INFO:
adding chunk, new array size ((1485204, 11))
2018-08-29 08:44:33,937:INFO:
adding chunk, new array size ((1488373, 11))
2018-08-29 08:44:36,061:INFO:
adding chunk, new array size ((1491276, 11))
2018-08-29 08:44:37,517:INFO:
adding chunk, new array size ((1494179, 11))
2018-08-29 08:44:3

In [ ]:
print(data)